In [1]:
from collections import deque

def print_move(source, target, disk):
    print(f"Move disk {disk} from {source} to {target}")

def hanoi(n, source, target, auxiliary):
    # Create a queue for BFS
    queue = deque([(source, target, auxiliary)])

    while queue:
        source, target, auxiliary = queue.popleft()

        # If all disks are in the target peg, we have solved the problem
        if len(target) == n:
            return source, target, auxiliary

        # Move a disk from source to auxiliary
        if source and (not auxiliary or source[-1] < auxiliary[-1]): # A && (~ B || C(ka last element) < B (ka last element) )
            print_move('source', 'auxiliary', source[-1])
            new_source, new_auxiliary = source[:-1], auxiliary + [source[-1]]
            queue.append((new_source, target, new_auxiliary))

        # Move a disk from source to target
        if source and (not target or source[-1] < target[-1]):
            print_move('source', 'target', source[-1])
            new_source, new_target = source[:-1], target + [source[-1]]
            queue.append((new_source, new_target, auxiliary))

        # Move a disk from auxiliary to target
        if auxiliary and (not target or auxiliary[-1] < target[-1]):
            print_move('auxiliary', 'target', auxiliary[-1])
            new_auxiliary, new_target = auxiliary[:-1], target + [auxiliary[-1]]
            queue.append((source, new_target, new_auxiliary))

# Test the function
source = [3, 2, 1]
target = []
auxiliary = []
hanoi(3, source, target, auxiliary)

Move disk 1 from source to auxiliary
Move disk 1 from source to target
Move disk 2 from source to target
Move disk 1 from auxiliary to target
Move disk 2 from source to auxiliary
Move disk 1 from auxiliary to target
Move disk 2 from source to auxiliary
Move disk 3 from source to auxiliary


In [2]:
from collections import deque

# Define the initial state
initial_state = (3, 3, 1)  # (missionaries on left, cannibals on left, boat position)

# Define the goal state
goal_state = (0, 0, 0)  # (all missionaries on right, all cannibals on right, boat on right)

# Function to check if a state is valid
def is_valid(state):
    missionaries_left, cannibals_left, boat = state
    missionaries_right = 3 - missionaries_left
    cannibals_right = 3 - cannibals_left
    
    # Check if missionaries are outnumbered by cannibals
    if missionaries_left < cannibals_left and missionaries_left > 0:
        return False
    if missionaries_right < cannibals_right and missionaries_right > 0:
        return False
    
    return True

# Function to generate next valid states
def next_states(state):
    possible_moves = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]
    moves = []
    missionaries_left, cannibals_left, boat = state
    
    if boat == 1:
        for move in possible_moves:
            missionaries_new = missionaries_left - move[0]
            cannibals_new = cannibals_left - move[1]
            if 0 <= missionaries_new <= 3 and 0 <= cannibals_new <= 3 and is_valid((missionaries_new, cannibals_new, 0)):
                moves.append((missionaries_new, cannibals_new, 0))
    else:
        for move in possible_moves:
            missionaries_new = missionaries_left + move[0]
            cannibals_new = cannibals_left + move[1]
            if 0 <= missionaries_new <= 3 and 0 <= cannibals_new <= 3 and is_valid((missionaries_new, cannibals_new, 1)):
                moves.append((missionaries_new, cannibals_new, 1))
    return moves

# Breadth-first search to find solution
def bfs():
    visited = set()
    queue = deque([(initial_state, [])])  # Queue of states and their path
    
    while queue:
        state, path = queue.popleft()
        visited.add(state)
        
        if state == goal_state:
            return path
        
        for next_state in next_states(state):
            if next_state not in visited:
                queue.append((next_state, path + [next_state]))
                visited.add(next_state)
    
    return None

# Find solution
solution = bfs()

# Print solution
if solution:
    print("Solution found:")
    for i, state in enumerate(solution):
        print(f"Step {i + 1}: {state}")
else:
    print("No solution found.")

Solution found:
Step 1: (3, 1, 0)
Step 2: (3, 2, 1)
Step 3: (3, 0, 0)
Step 4: (3, 1, 1)
Step 5: (1, 1, 0)
Step 6: (2, 2, 1)
Step 7: (0, 2, 0)
Step 8: (0, 3, 1)
Step 9: (0, 1, 0)
Step 10: (1, 1, 1)
Step 11: (0, 0, 0)


In [3]:

class State:
    def __init__(self, m, c, b):
        self.m = m  # number of missionaries on the starting bank
        self.c = c  # number of cannibals on the starting bank
        self.b = b  # 1 if boat is on starting bank, 0 if on other bank

    def is_valid(self):
        if self.m < 0 or self.c < 0 or self.m > 3 or self.c > 3:
            return False
        if self.m < self.c and self.m > 0:
            return False
        if 3 - self.m < 3 - self.c and 3 - self.m > 0:
            return False
        return True

    def is_goal(self):
        return self.m == 0 and self.c == 0 and self.b == 0

    def __eq__(self, other):
        return self.m == other.m and self.c == other.c and self.b == other.b

    def __hash__(self):
        return hash((self.m, self.c, self.b))


def get_successors(state):
    successors = []
    if state.b == 1:
        successors += [State(state.m - 1, state.c, 0),
                       State(state.m - 2, state.c, 0),
                       State(state.m, state.c - 1, 0),
                       State(state.m, state.c - 2, 0),
                       State(state.m - 1, state.c - 1, 0)]
    else:
        successors += [State(state.m + 1, state.c, 1),
                       State(state.m + 2, state.c, 1),
                       State(state.m, state.c + 1, 1),
                       State(state.m, state.c + 2, 1),
                       State(state.m + 1, state.c + 1, 1)]
    return [s for s in successors if s.is_valid()]


def dfs(state, visited, path):
    visited.add(state)
    path.append(state)
    if state.is_goal():
        return True
    for successor in get_successors(state):
        if successor not in visited:
            if dfs(successor, visited, path):
                return True
    path.pop()
    return False


def solve():
    initial_state = State(3, 3, 1)
    visited = set()
    path = []
    if dfs(initial_state, visited, path):
        for i, state in enumerate(path):
            print(f"Step {i + 1}: (M={state.m}, C={state.c}, B={'left' if state.b == 1 else 'right'})")
    else:
        print("No solution found.")


if __name__ == "__main__":
    solve()


Step 1: (M=3, C=3, B=left)
Step 2: (M=3, C=1, B=right)
Step 3: (M=3, C=2, B=left)
Step 4: (M=3, C=0, B=right)
Step 5: (M=3, C=1, B=left)
Step 6: (M=1, C=1, B=right)
Step 7: (M=2, C=2, B=left)
Step 8: (M=0, C=2, B=right)
Step 9: (M=0, C=3, B=left)
Step 10: (M=0, C=1, B=right)
Step 11: (M=1, C=1, B=left)
Step 12: (M=0, C=0, B=right)


In [4]:
from random import choice
from math import inf

board = [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]

def Gameboard(board):
    chars = {1: 'X', -1: 'O', 0: ' '}
    for x in board:
        for y in x:
            ch = chars[y]
            print(f'| {ch} |', end='')
        print('\n' + '---------------')
    print('===============')

def Clearboard(board):
    for x, row in enumerate(board):
        for y, col in enumerate(row):
            board[x][y] = 0

def winningPlayer(board, player):
    conditions = [[board[0][0], board[0][1], board[0][2]],
                     [board[1][0], board[1][1], board[1][2]],
                     [board[2][0], board[2][1], board[2][2]],
                     [board[0][0], board[1][0], board[2][0]],
                     [board[0][1], board[1][1], board[2][1]],
                     [board[0][2], board[1][2], board[2][2]],
                     [board[0][0], board[1][1], board[2][2]],
                     [board[0][2], board[1][1], board[2][0]]]

    if [player, player, player] in conditions:
        return True

    return False

def gameWon(board):
    return winningPlayer(board, 1) or winningPlayer(board, -1)

def printResult(board):
    if winningPlayer(board, 1):
        print('X has won! ' + '\n')

    elif winningPlayer(board, -1):
        print('O\'s have won! ' + '\n')

    else:
        print('Draw' + '\n')

def blanks(board):
    blank = []
    for x, row in enumerate(board):
        for y, col in enumerate(row):
            if board[x][y] == 0:
                blank.append([x, y])

    return blank

def boardFull(board):
    if len(blanks(board)) == 0:
        return True
    return False

def setMove(board, x, y, player):
    board[x][y] = player

def playerMove(board):
    e = True
    moves = {1: [0, 0], 2: [0, 1], 3: [0, 2],
             4: [1, 0], 5: [1, 1], 6: [1, 2],
             7: [2, 0], 8: [2, 1], 9: [2, 2]}
    while e:
        try:
            move = int(input('Enter a number between 1-9: '))
            if move < 1 or move > 9:
                print('Invalid Move! Try again!')
            elif not (moves[move] in blanks(board)):
                print('Invalid Move! Try again!')
            else:
                setMove(board, moves[move][0], moves[move][1], 1)
                Gameboard(board)
                e = False
        except(KeyError, ValueError):
            print('Enter a number!')

def getScore(board):
    if winningPlayer(board, 1):
        return 10

    elif winningPlayer(board, -1):
        return -10

    else:
        return 0

def abminimax(board, depth, alpha, beta, player):
    row = -1
    col = -1
    if depth == 0 or gameWon(board):
        return [row, col, getScore(board)]

    else:
        for cell in blanks(board):
            setMove(board, cell[0], cell[1], player)
            score = abminimax(board, depth - 1, alpha, beta, -player)
            if player == 1:
                # X is always the max player
                if score[2] > alpha:
                    alpha = score[2]
                    row = cell[0]
                    col = cell[1]
            else:
                if score[2] < beta:
                    beta = score[2]
                    row = cell[0]
                    col = cell[1]
            setMove(board, cell[0], cell[1], 0)

            if alpha >= beta:
                break

        if player == 1:
            return [row, col, alpha]
        else:
            return [row, col, beta]

def o_comp(board):
    if len(blanks(board)) == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
        setMove(board, x, y, -1)
        Gameboard(board)

    else:
        result = abminimax(board, len(blanks(board)), -inf, inf, -1)
        setMove(board, result[0], result[1], -1)
        Gameboard(board)

def x_comp(board):
    if len(blanks(board)) == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
        setMove(board, x, y, 1)
        Gameboard(board)
    else:
        result = abminimax(board, len(blanks(board)), -inf, inf, 1)
        setMove(board, result[0], result[1], 1)
        Gameboard(board)

def makeMove(board, player, mode):
    if mode == 1:
        if player == 1:
            playerMove(board)
        else:
            o_comp(board)
    else:
        if player == 1:
            o_comp(board)
        else:
            x_comp(board)

def pvc():
    while True:
        try:
            order = int(input('Enter to play 1st or 2nd: '))
            if not (order == 1 or order == 2):
                print('Please pick 1 or 2')
            else:
                break
        except(KeyError, ValueError):
            print('Enter a number')

    Clearboard(board)
    if order == 2:
        currentPlayer = -1
    else:
        currentPlayer = 1

    while not (boardFull(board) or gameWon(board)):
        makeMove(board, currentPlayer, 1)
        currentPlayer *= -1

    printResult(board)

print("TIC-TAC-TOE using MINIMAX with ALPHA-BETA Pruning")
pvc()

TIC-TAC-TOE using MINIMAX with ALPHA-BETA Pruning
Enter to play 1st or 2nd: 1
Enter a number between 1-9: 3
|   ||   || X |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   || X |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Enter a number between 1-9: 1
| X ||   || X |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
| X || O || X |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Enter a number between 1-9: 8
| X || O || X |
---------------
|   || O ||   |
---------------
|   || X ||   |
---------------
| X || O || X |
---------------
| O || O ||   |
---------------
|   || X ||   |
---------------
Enter a number between 1-9: 6
| X || O || X |
---------------
| O || O || X |
---------------
|   || X ||   |
---------------
| X || O || X |
---------------
| O || O || X |
---------------
|   || X || O |
---------------
Enter a number between 1-9: 6
Inva

In [7]:
import random 

def print_board(board):
    for row in board:
        print("|" .join(row))
        print("_" * 10)
        
        
def is_winner(board,player):
    for row in board:
        if all(cell == player for cell in row):
            return True
        
    for col in range(3):
        if all(board[row][col] == player for row in range(3)):
            return True
                   
    if all(board[i][i] == player for i in range(3)) or all(board[i][2-i] == player for i in range(3)):
        return True
    return False
                   

def is_board_full(board):
    return all(cell != ' ' for row in board for cell in row)
                   
                   
                   
def get_user_move():
    while True:
        try:
            move = int(input("Enter a number from (0-9)"))
            if 1 <= move <= 9:
                return move
            else:
                print("Invalid input , please try again")
                
        except ValueError:
            print("Enter a numeric value between (0-9)")
            

def calculate_computer_move(board,player_symbol,computer_symbol):
    magic_square = [
        [8,3,4],
        [1,5,9],
        [6.7,2]
    ]
    
    
    empty_cells = [(i,j) for i in range(3) for j in range(3) if board[i][j] == ' ']
    
    for i,j in empty_cells:
        temp_board = [row[:] for row in board]
        temp_board[i][j] = computer_symbol
        if is_winner(temp_board,computer_symbol):
            return i * 3 + j + 1
        
    for i,j in empty_cells:
        temp_board = [row[:] for row in board]
        temp_board[i][j] = player_symbol
        if is_winner(temp_board,computer_symbol):
            return i * 3 + j + 1 
        
    return random.choice(empty_cells)[0]*3 + random.choice(empty_cells)[1] + 1


def play_tic_tac_toe():
    board = [[' ' for i in range(3)]for i in range(3)]
    user_symbol, computer_symbol = "X", "O"
    
    print("welcome to tic tac toe\n")
    print_board(board)
    
    for move_num in range (1,10):
        
        current_player = user_symbol if move_num % 2 == 1 else computer_symbol
        
        if current_player == user_symbol:
            user_move = get_user_move()
            row, col = divmod(user_move-1, 3)
        else:
            computer_move = calculate_computer_move(board, user_symbol, computer_symbol)
            row, col = divmod(computer_move-1,3)
            print(f"Computer chooses position {computer_move}")
            
        while board[row][col] != ' ':
            print("ERROR, position already taken choose a different position")
            
            if current_player == user_symbol:
                user_move = get_user_move()
                row, col = divmod(user_move-1, 3)
            else:
                computer_move = calculate_computer_move(board, user_symbol, computer_symbol)
                row, col = divmod(computer_move-1,3)
            
            
        
        if current_player == user_symbol:
            board[row][col] = user_symbol
        else:
            board[row][col] = computer_symbol
        print_board(board)
        
        if is_winner(board,current_player):
            print(f"{current_player} wins")
            return
        
        if is_board_full(board):
            print("its a tie")
            return
        
play_tic_tac_toe()    
            
            
    
            
        
        
 

                                                              

welcome to tic tac toe

 | | 
__________
 | | 
__________
 | | 
__________
Enter a number from (0-9)2
 |X| 
__________
 | | 
__________
 | | 
__________
Computer chooses position 6
 |X| 
__________
 | |O
__________
 | | 
__________
Enter a number from (0-9)9
 |X| 
__________
 | |O
__________
 | |X
__________
Computer chooses position 3
 |X|O
__________
 | |O
__________
 | |X
__________
Enter a number from (0-9)5
 |X|O
__________
 |X|O
__________
 | |X
__________
Computer chooses position 5
ERROR, position already taken choose a different position
 |X|O
__________
 |X|O
__________
 |O|X
__________
Enter a number from (0-9)1
X|X|O
__________
 |X|O
__________
 |O|X
__________
X wins


In [16]:



from nqueens import *

qq=Queen(5)
qq.pprint()

N =  5
No of solutions possible =  10
Solution space =  [[[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0]], [[1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0]], [[0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]], [[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0]], [[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]], [[0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0]], [[0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0]], [[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0]], [[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0]], [[0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0]]]


In [15]:
pip install nqueens


Note: you may need to restart the kernel to use updated packages.
